In [1]:
# Importing the libraries 

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

# Ignore harmless warnings 

import warnings 
warnings.filterwarnings("ignore")

# Set to display all the columns in dataset

pd.set_option("display.max_columns", None)

# Import psql to run queries 

import pandasql as psql

In [3]:
# Load the data from sheet1

MHealth = pd.read_csv(r"C:\Users\manso\Downloads\letter-recognition2.csv", header=0)

# Copy 20% data (stratified split on target variable) to new dataset 

MHealth = MHealth.groupby('letter', group_keys=False).apply(lambda x: x.sample(frac=0.2))

# Display the shape

MHealth.shape

(4002, 17)

In [4]:
# Re set the index

MHealth = MHealth.reset_index(drop=True)

# Copy to back-up file

MHealth_bk = MHealth.copy()

# display information of the dataset

MHealth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4002 entries, 0 to 4001
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  4002 non-null   object
 1   xbox    4002 non-null   int64 
 2   ybox    4002 non-null   int64 
 3   width   4002 non-null   int64 
 4   height  4002 non-null   int64 
 5   onpix   4002 non-null   int64 
 6   xbar    4002 non-null   int64 
 7   ybar    4002 non-null   int64 
 8   x2bar   4002 non-null   int64 
 9   y2bar   4002 non-null   int64 
 10  xybar   4002 non-null   int64 
 11  x2ybar  4002 non-null   int64 
 12  xy2bar  4002 non-null   int64 
 13  xedge   4002 non-null   int64 
 14  xedgey  4002 non-null   int64 
 15  yedge   4002 non-null   int64 
 16  yedgex  4002 non-null   int64 
dtypes: int64(16), object(1)
memory usage: 531.6+ KB


In [5]:
# Identify the missing values

MHealth.isnull().sum()

letter    0
xbox      0
ybox      0
width     0
height    0
onpix     0
xbar      0
ybar      0
x2bar     0
y2bar     0
xybar     0
x2ybar    0
xy2bar    0
xedge     0
xedgey    0
yedge     0
yedgex    0
dtype: int64

In [6]:
# Display Unique values of variables

MHealth.nunique()

letter    26
xbox      15
ybox      16
width     16
height    16
onpix     15
xbar      16
ybar      16
x2bar     16
y2bar     16
xybar     16
x2ybar    16
xy2bar    16
xedge     16
xedgey    14
yedge     15
yedgex    14
dtype: int64

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in MHealth.columns:
    if MHealth[col].dtypes == 'object':
        MHealth[col] = le.fit_transform(MHealth[col])

In [10]:
MHealth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4002 entries, 0 to 4001
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   letter  4002 non-null   int32
 1   xbox    4002 non-null   int64
 2   ybox    4002 non-null   int64
 3   width   4002 non-null   int64
 4   height  4002 non-null   int64
 5   onpix   4002 non-null   int64
 6   xbar    4002 non-null   int64
 7   ybar    4002 non-null   int64
 8   x2bar   4002 non-null   int64
 9   y2bar   4002 non-null   int64
 10  xybar   4002 non-null   int64
 11  x2ybar  4002 non-null   int64
 12  xy2bar  4002 non-null   int64
 13  xedge   4002 non-null   int64
 14  xedgey  4002 non-null   int64
 15  yedge   4002 non-null   int64
 16  yedgex  4002 non-null   int64
dtypes: int32(1), int64(16)
memory usage: 516.0 KB


In [12]:
# Identify the Independent and Target variables

IndepVar = []
for col in MHealth.columns:
    if col != 'letter':
        IndepVar.append(col)

TargetVar = 'letter'

x = MHealth[IndepVar]
y = MHealth[TargetVar]

In [13]:
# Split the data into train and test (random sampling)

from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Multi Label KNN Algorithm

[[46  0  0  0  0  1  0  0  0  0  0  0  3  1  0  0  0  0  0  0  2  0  0  0
   0  0]
 [ 1 52  0  1  1  1  1  2  0  0  2  0  1  0  0  0  0  1  4  1  0  1  0  1
   0  1]
 [ 0  0 36  0  2  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  3  0 49  0  1  1  8  0  0  0  1  0  1  1  1  1  0  0  2  0  0  0  0
   1  0]
 [ 0  2  3  0 36  1  5  0  0  0  8  1  0  0  0  0  1  0  2  0  0  0  0  1
   0  1]
 [ 0  0  0  0  2 41  0  0  2  0  0  0  0  0  0  7  0  0  2  1  0  1  0  0
   0  0]
 [ 0  0  3  0  0  0 37  0  0  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0
   0  0]
 [ 0  1  0  0  0  0  0 29  0  1  2  0  1  2  1  1  0  1  0  0  0  0  0  0
   3  0]
 [ 0  0  0  0  0  0  0  0 36  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  0  0  0  0  0  0  0  2 36  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  1  0  0  0  0  0  0  0  0 30  0  0  0  0  0  0  1  0  0  0  0  0  7
   0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 41  0  0  0  0  0  0  0  0  0  0  0  2
   0  0]
 [ 1

In [14]:
# To build the 'Multi Label Extra Trees' model with random sampling

from sklearn.neighbors import KNeighborsClassifier

# Create an object for model

ModelKNN = KNeighborsClassifier(n_neighbors=5)

# Train the model with training data

ModelKNN = ModelKNN.fit(x_train,y_train)

# Predict the model with test data set

y_pred = ModelKNN.predict(x_test)
y_pred_prob = ModelKNN.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from math import sqrt

print(confusion_matrix(y_pred, y_test)) # Verticle is actual values & horizontal is predicted values

# Actual and predicted classes

lst_actual_class = y_test
lst_predicted_class = y_pred
lst_predicted_prob_class = y_pred_prob

# Class = Label 0-12

lst_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# Compute multi-class confusion matrix

arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results

model_acc = [];
model_recall = [];
model_prec = [];
model_fscore = [];
model_spec = [];
model_bal_acc = [];
model_mcc = [];
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Print Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fn = arr_data[0][1]
    tn = arr_data[0][0]
    fp = arr_data[1][0]
    
    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);
    
    mx = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(mx), 3)
    model_acc.append(accuracy);
    model_prec.append(precision);
    model_recall.append(sensitivity);
    model_fscore.append(f1Score);
    model_spec.append(specificity);
    model_bal_acc.append(balanced_accuracy);
    model_mcc.append(MCC);
    
    print("TP={0}, FN={1}, TN={2}, FP={3}".format(tp, fn, tn, fp));
    print("Accuracy: {0}".format(accuracy));    # Accuracy score
    print("Precision: {0}".format(precision)); # Precision score
    print("Sensitivity: {0}".format(sensitivity)); # Recall score
    print("F1-Score: {0}".format(f1Score)); # F1 score
    print("Specificity: {0}".format(specificity)); # True Nagative Rate
    print("Balanced Accuracy: {0}".format(balanced_accuracy)); # Balance accuracy score
    print("MCC: {0}\n".format(MCC)); # Matthews Correlation Coefficient
    
# OVERALL - FINAL PREDICTION PERFORMANCE
   
# importing mean()
   
from statistics import mean
import math

print("Overall Performance Prediction:");
print("Accuracy: {0}%".format(round(mean(model_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(model_prec)*100, 4)));
print("Recall or Sensitivity: {0}%".format(round(mean(model_recall)*100, 4)));
print("F1-Score: {0}".format(round(mean(model_fscore), 4)));
print("Specificity or True Nagative Rate: {0}%".format(round(mean(model_spec)*100, 4)));
print("Balanced Accuracy: {0}%\n".format(round(mean(model_bal_acc)*100, 4)));
print("MCC: {0}\n".format(round(mean(model_mcc), 4)))

#
# ROC curve for Multi classes 

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, roc_auc_score

fpr = {}
tpr = {}
thresh ={}

n_class = 13

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(lst_actual_class, lst_predicted_prob_class[:,i], pos_label=i)
    
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='brown', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class 2 vs Rest')
plt.plot(fpr[3], tpr[3], linestyle='--',color='red', label='Class 3 vs Rest')
plt.plot(fpr[4], tpr[4], linestyle='--',color='purple', label='Class 4 vs Rest')
plt.plot(fpr[5], tpr[5], linestyle='--',color='yellow', label='Class 5 vs Rest')
plt.plot(fpr[6], tpr[6], linestyle='--',color='green', label='Class 6 vs Rest')
plt.plot(fpr[7], tpr[7], linestyle='--',color='blue', label='Class 7 vs Rest')
plt.plot(fpr[8], tpr[8], linestyle='--',color='magenta', label='Class 8 vs Rest')
plt.plot(fpr[9], tpr[9], linestyle='--',color='black', label='Class 9 vs Rest')
plt.plot(fpr[10], tpr[10], linestyle='--',color='red', label='Class 10 vs Rest')
plt.plot(fpr[11], tpr[11], linestyle='--',color='green', label='Class 11 vs Rest')
plt.plot(fpr[12], tpr[12], linestyle='--',color='orange', label='Class 12 vs Rest')

plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')

# ROC AUC score - one-vs-one (OvO) algorithm computes the average of the ROC AUC scores for each class against 
# all other classes. 

print('roc_auc_score:', round(roc_auc_score(lst_actual_class, lst_predicted_prob_class, multi_class='ovo', 
                                            average='weighted'),3))
#

In [15]:
# Results

pd.set_option("display.max_rows", None)

PredResults = pd.DataFrame({'letter':lst_actual_class, 'letter':lst_predicted_class})

# Merge two Dataframes on index of both the dataframes

TestDataResults = x_test.merge(PredResults, left_index=True, right_index=True)
TestDataResults.sample(10)

,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex,letter
227,4,10,6,8,7,8,7,4,7,6,6,6,6,8,6,10,7
283,2,0,2,1,1,7,7,7,5,7,6,7,1,8,7,8,7
1200,4,5,6,4,4,7,8,3,6,10,7,7,5,9,4,8,21
794,2,3,4,2,1,6,10,3,5,13,6,5,1,9,2,7,4
1192,2,4,3,3,2,8,8,6,6,7,5,7,3,8,3,7,16
371,3,8,4,6,2,4,8,7,9,9,9,13,1,9,4,9,12
1113,8,11,12,8,10,9,6,3,7,10,5,8,6,8,5,8,18
720,6,8,8,7,8,5,10,4,5,8,7,9,4,10,9,7,1
1078,4,8,5,6,2,8,6,8,8,6,6,9,2,7,6,10,12
124,4,9,7,7,4,12,3,2,2,9,2,9,2,6,2,8,9
